In [1]:
from mxnet import ndarray as nd
from mxnet import autograd
from mxnet import gluon

num_inputs = 2
num_examples = 1000

true_w = [2, -3.4]
true_b = 4.2

X = nd.random_normal(shape=(num_examples, num_inputs))
y = true_w[0] * X[:, 0] + true_w[1] * X[:, 1] + true_b
y += .01 * nd.random_normal(shape=y.shape)

In [2]:
batch_size = 10
dataset = gluon.data.ArrayDataset(X, y)
data_iter = gluon.data.DataLoader(dataset, batch_size, shuffle=True)

In [3]:
for data, label in data_iter:
    print(data, label)
    break


[[-0.62632275  0.40309143]
 [-0.13549392 -1.49542165]
 [-0.34295559 -0.24174212]
 [ 0.278135    1.8619597 ]
 [ 0.40446803  0.15614767]
 [ 0.08757513  0.30246663]
 [ 1.30263245  0.84225523]
 [ 0.62555301  0.35536364]
 [-0.51837742 -1.1084621 ]
 [ 2.18415785 -0.70068365]]
<NDArray 10x2 @cpu(0)> 
[  1.56013811   9.01445675   4.35061979  -1.58218455   4.49080896
   3.33912539   3.93966675   4.23716211   6.93560076  10.93650341]
<NDArray 10 @cpu(0)>


In [5]:
# Sequential会依次执行每一层，并将前一层的输出，
# 作为输入提供给后面的层。首先我们定义一个空的模型
net = gluon.nn.Sequential()

In [6]:
net.add(gluon.nn.Dense(1))

In [7]:
net.initialize()

In [8]:
square_loss = gluon.loss.L2Loss()

In [9]:
trainer = gluon.Trainer(
    net.collect_params(), 'sgd', {'learning_rate': 0.1})

In [10]:
epochs = 5
batch_size = 10
for e in range(epochs):
    total_loss = 0
    for data, label in data_iter:
        with autograd.record():
            output = net(data)
            loss = square_loss(output, label)
        loss.backward()
        trainer.step(batch_size)
        total_loss += nd.sum(loss).asscalar()
    print("Epoch %d, average loss: %f" % (e, total_loss/num_examples))

Epoch 0, average loss: 0.890537
Epoch 1, average loss: 0.000047
Epoch 2, average loss: 0.000047
Epoch 3, average loss: 0.000047
Epoch 4, average loss: 0.000046


In [15]:
dense = net[0]
dense, dense.weight.data()

(Dense(2 -> 1, linear), 
 [[ 1.99911702 -3.40092492]]
 <NDArray 1x2 @cpu(0)>)

In [16]:
true_b, dense.bias.data()

(4.2, 
 [ 4.19964743]
 <NDArray 1 @cpu(0)>)